In [1]:
import optuna
import joblib
import numpy as np
import pandas as pd
import sys
from project_resources.import_utils import NotebookFinder
sys.meta_path.append(NotebookFinder())
from project_resources.cytochrome_P450 import HyperparamTuner, mol_fts, abs_file_path

importing Jupyter notebook from C:\Users\Lukas\Documents\datacytochromy\project_resources\cytochrome_P450.ipynb


In [2]:
model_identifiers = ["linear", "KRR", "GB", "RF", "ANN"]
isozymes = ["3A4", "RLM", "HLC"]
data_splits = ["train", "test"]
splitters = ["rand", "scaff"]
rel_paths = {
    "3A4_source": r"project_resources/ChEMBL_3A4.csv",
    "3A4_sep": ";",
    "3A4": r"project_resources/3A4.csv",
    "3A4_train_scaff": r"project_resources/base_splits/scaffold_splitter/3A4_train.csv",
    "3A4_test_scaff": r"project_resources/base_splits/scaffold_splitter/3A4_test.csv",
    "3A4_train_rand": r"project_resources/base_splits/random/3A4_train.csv",
    "3A4_test_rand": r"project_resources/base_splits/random/3A4_test.csv",
    "3A4_jazzy_train_rand": r"project_resources/jazzy_splits/random/3A4_train.csv",
    "3A4_jazzy_test_rand": r"project_resources/jazzy_splits/random/3A4_test.csv",
    "3A4_jazzy_train_scaff": r"project_resources/jazzy_splits/scaffold_splitter/3A4_train.csv",
    "3A4_jazzy_test_scaff": r"project_resources/jazzy_splits/scaffold_splitter/3A4_test.csv",

    "RLM_source": r"project_resources/AID_1508591_datatable_all.csv",
    "RLM_sep": ",",
    "RLM": r"project_resources/RLM.csv",
    "RLM_train_scaff": r"project_resources/base_splits/scaffold_splitter/RLM_train.csv",
    "RLM_test_scaff": r"project_resources/base_splits/scaffold_splitter/RLM_test.csv",
    "RLM_train_rand": r"project_resources/base_splits/random/RLM_train.csv",
    "RLM_test_rand": r"project_resources/base_splits/random/RLM_test.csv",
    "RLM_jazzy_train_rand": r"project_resources/jazzy_splits/random/RLM_train.csv",
    "RLM_jazzy_test_rand": r"project_resources/jazzy_splits/random/RLM_test.csv",
    "RLM_jazzy_train_scaff": r"project_resources/jazzy_splits/scaffold_splitter/RLM_train.csv",
    "RLM_jazzy_test_scaff": r"project_resources/jazzy_splits/scaffold_splitter/RLM_test.csv",


    "HLC_source": r"project_resources/AID_1508603_datatable_all.csv",
    "HLC_sep": ",",
    "HLC": r"project_resources/HLC.csv",
    "HLC_train_scaff": r"project_resources/base_splits/scaffold_splitter/HLC_train.csv",
    "HLC_test_scaff": r"project_resources/base_splits/scaffold_splitter/HLC_test.csv",
    "HLC_train_rand": r"project_resources/base_splits/random/HLC_train.csv",
    "HLC_test_rand": r"project_resources/base_splits/random/HLC_test.csv",
    "HLC_jazzy_train_rand": r"project_resources/jazzy_splits/random/HLC_train.csv",
    "HLC_jazzy_test_rand": r"project_resources/jazzy_splits/random/HLC_test.csv",
    "HLC_jazzy_train_scaff": r"project_resources/jazzy_splits/scaffold_splitter/HLC_train.csv",
    "HLC_jazzy_test_scaff": r"project_resources/jazzy_splits/scaffold_splitter/HLC_test.csv"
}
# sampler - a method used to generate new sets of hyperparameters in each iteration of the optimization process
samplers = {
    'RandomSampler': optuna.samplers.RandomSampler,          # Sampler that selects hyperparameters randomly from the search space.
    'GridSampler': optuna.samplers.GridSampler,              # Sampler that performs a grid search over the hyperparameter space.
    'TPESampler': optuna.samplers.TPESampler,                # Sampler that uses a tree-structured Parzen estimator to model the objective function and sample new points from the search space.
    'CmaEsSampler': optuna.samplers.CmaEsSampler,            # Sampler that uses the Covariance Matrix Adaptation Evolution Strategy algorithm to efficiently search the hyperparameter space.
    'NSGAIISampler': optuna.samplers.NSGAIISampler,          # Multi-objective evolutionary algorithm that generates new samples using non-dominated sorting and crowding distance selection.
    'QMCSampler': optuna.samplers.QMCSampler,                # Quasi-Monte Carlo sampler that uses low-discrepancy sequences to sample the search space in a more efficient and evenly distributed way than random sampling.
    'BoTorchSampler': optuna.integration.BoTorchSampler,     # Sampler that leverages the BoTorch library for Bayesian optimization and can handle both continuous and categorical hyperparameters.
    'BruteForceSampler': optuna.samplers.BruteForceSampler,  # Sampler that exhaustively evaluates all possible combinations of hyperparameters in the search space.
}
# pruner - a technique used to eliminate unpromising trials during the course of hyperparameter optimization.
pruners = {
    'BasePruner': optuna.pruners.BasePruner,                            # This is the base class for all pruning strategies in Optuna. It provides a skeleton for implementing custom pruning strategies.
    'MedianPruner': optuna.pruners.MedianPruner,                        # A pruner that prunes unpromising trials that have median objective values, as determined in previous steps.
    'SuccessiveHalvingPruner': optuna.pruners.SuccessiveHalvingPruner,  # This pruner repeatedly splits trials into halves, discarding the lower performing half at each iteration.
    'HyperbandPruner': optuna.pruners.HyperbandPruner,                  # This pruner implements the Hyperband algorithm, which selects promising trials and runs them with different resource allocation schemes to determine the best one.
    'PercentilePruner': optuna.pruners.PercentilePruner,                # A pruner that prunes unpromising trials based on their percentile rank relative to all completed trials.
    'NopPruner': optuna.pruners.NopPruner,                              # A pruner that does nothing and does not prune any trials.
    'ThresholdPruner': optuna.pruners.ThresholdPruner,                  # This pruner prunes trials that have not reached a certain level of performance (i.e., objective value).
    'PatientPruner': optuna.pruners.PatientPruner,                      # This pruner prunes trials that do not show improvement over a certain number of steps (or epochs).
}
halflives = {}
mol_features = {}

In [3]:
# load Jazzy features from csv files or generate the features and save them
for splitter in splitters:
    print("\n")
    print(splitter)
    mol_features[splitter] = {}
    halflives[splitter] = {}
    for isozyme in isozymes:
        mol_features[splitter][isozyme] = {}
        halflives[splitter][isozyme] = {}
        for split in data_splits:
            print(isozyme, split)

            try:
                df = pd.read_csv(rel_paths[f"{isozyme}_jazzy_{split}_{splitter}"])
                cols = df.columns
                data = {}  # all data from csv file (i.e. mol indexes, smiles, half-lives and features)
                for col in cols:
                    data[col] = list(df[col])
                nan_idxs = np.argwhere(np.isnan(data["dgtot"]))
                nan_idxs = [int(idx) for idx in nan_idxs]
                data_clumped = []  # same as data, but in the form [[idx1, smi1, thalf1, fts1], [idx2, smi2, thalf2, fts2],...]]
                for col in cols:
                    for i, foo in zip(range(len(data[col])), data[col]):
                        if len(data_clumped) < i+1:
                            data_clumped.append([])
                        data_clumped[i].append(foo)

                # remove all mols for which Jazzy features generation wasn't successful
                num_pops = 0
                for nan_idx in nan_idxs:
                    data_clumped.pop(nan_idx - num_pops)
                    num_pops += 1
                    print(f"     removed index {nan_idx} corresponding to NaN in {split}")
                print(f"     {len(data_clumped)}, {data_clumped[0]}")

                # filter out only the features
                mol_features[splitter][isozyme][split] = np.array([feature[3:11] for feature in data_clumped])
                halflives[splitter][isozyme][split] = np.array([feature[2] for feature in data_clumped])
                contains_nan = np.any(np.isnan(mol_features[splitter][isozyme][split]))
                print(f"     {mol_features[splitter][isozyme][split][0]}")
                print(f"     {isozyme} mol_features {split} contains NaN: {contains_nan}")

            except FileNotFoundError:
                df = pd.read_csv(rel_paths[f"{isozyme}_{split}_{splitter}"])
                mol_idx = df["index"]
                smiles = df["smiles"]
                halflife = df["half-life"]
                mol_features[splitter][isozyme] = mol_fts(smiles, isozyme)  # calculate mol features for list of smiles
                mol_fts_df = pd.DataFrame()
                mol_fts_df["index"] = mol_idx
                mol_fts_df["smiles"] = smiles
                mol_fts_df["half-life"] = halflife
                present_fts = [feature for feature in mol_features[splitter][isozyme][0]]  # list of all mol features

                # write the value of a given feature for each molecule into df
                for feature in present_fts:
                    single_feature_list = []
                    for i in range(len(mol_features[splitter][isozyme])):
                        if mol_features[splitter][isozyme][i] == mol_features[splitter][isozyme][i]:
                            single_feature_list.append(mol_features[splitter][isozyme][i][feature])
                        else:
                            single_feature_list.append(np.nan)
                    mol_fts_df[feature] = single_feature_list

                mol_fts_df.to_csv(abs_file_path(rel_paths[f"{isozyme}_jazzy_{split}_{splitter}"]), index=False)
                print(f'{rel_paths[f"{isozyme}_jazzy_{split}_{splitter}"]} was created successfully')



rand
3A4 train
     56, [63, 'CC(C)(O)c1cc(F)c2c(c1)C(=O)N(Cc1ccc(Cl)cn1)[C@@]2(OCC1(O)CC1)c1ccc(Cl)cc1', 6.0, 13.1753, 1.4177, 11.3044, -7.3559, -125.8906, -119.4889]
     [  13.1753    1.4177   11.3044   -7.3559 -125.8906 -119.4889]
     3A4 mol_features train contains NaN: False
3A4 test
     14, [23, 'Cc1ncsc1-c1ccc([C@H](CC(=O)NCCCCCCNC(=O)COc2c(-c3csc(N4CCOCC4)n3)ccc(F)c2F)NC(=O)[C@@H]2C[C@@H](O)CN2C(=O)[C@@H](NC(=O)C2(F)CC2)C(C)(C)C)cc1', 0.3767, 28.4327, 3.274, 23.4872, -4.0348, -286.1545, -257.0326]
     [  28.4327    3.274    23.4872   -4.0348 -286.1545 -257.0326]
     3A4 mol_features test contains NaN: False
RLM train
     removed index 1047 corresponding to NaN in train
     removed index 1517 corresponding to NaN in train
     2022, [158, 'O=c1cc(N2CCOCC2)oc2c1ccc1ccccc12', 30.0, 8.0486, 0.0, 5.2165, -13.5913, -63.7029, -67.7504]
     [  8.0486   0.       5.2165 -13.5913 -63.7029 -67.7504]
     RLM mol_features train contains NaN: False
RLM test
     removed index 174 c

In [5]:
sampler = samplers['TPESampler']
pruner = pruners["BasePruner"]
n_trials = 1
for splitter in splitters:
    if splitter == "rand":
        splitter_name = "random"
    else:
        splitter_name = "scaffold_splitter"

    for isozyme in isozymes:
        X_train = mol_features[splitter][isozyme]["train"]
        y_train = np.log(halflives[splitter][isozyme]["train"])  # natural log half-lives -> values are less spread out
        X_test = mol_features[splitter][isozyme]["test"]
        y_test = np.log(halflives[splitter][isozyme]["test"])
        for model_identifier in model_identifiers:
            print(splitter_name, isozyme, model_identifier)
            study = optuna.create_study(study_name=model_identifier, directions=['minimize'], pruner=pruner,
                                        storage=f"sqlite:///project_resources/optuna/jazzy/{splitter_name}/{isozyme}/db.{model_identifier}", load_if_exists=True)
            test = HyperparamTuner(model_identifier, X_train, y_train, X_test, y_test)
            study.optimize(test.objective, n_trials=n_trials, n_jobs=-1)  # catch=(ValueError,)
            joblib.dump(study, f"./project_resources/optuna/jazzy/{splitter_name}/{isozyme}/{model_identifier}.pkl")

[I 2023-10-10 22:05:55,404] Using an existing study with name 'linear' instead of creating a new one.


random 3A4 linear


[I 2023-10-10 22:05:55,479] Trial 1 finished with value: 1.4052157267487904 and parameters: {'alpha': 0.09756333251827164, 'l1_ratio': 0.2842031992425431}. Best is trial 0 with value: 1.4038627192898365.
[I 2023-10-10 22:05:55,609] A new study created in RDB with name: KRR


random 3A4 KRR


[I 2023-10-10 22:05:55,695] Trial 0 finished with value: 1.3804652303052911 and parameters: {'alpha': 0.38984005784428544, 'gamma': 7.247000271055202e-15, 'kernel': 'rbf'}. Best is trial 0 with value: 1.3804652303052911.
[I 2023-10-10 22:05:55,822] A new study created in RDB with name: GB


...

[I 2023-10-10 22:06:47,334] Trial 0 finished with value: 0.5036686982461732 and parameters: {'n_estimators': 10, 'learning_rate': 0.2048964546603394, 'max_depth': 5}. Best is trial 0 with value: 0.5036686982461732.
[I 2023-10-10 22:06:47,480] A new study created in RDB with name: RF


scaffold_splitter HLC RF


[I 2023-10-10 22:06:48,751] Trial 0 finished with value: 0.4514864164705113 and parameters: {'n_estimators': 50, 'max_features': 'auto', 'max_depth': None}. Best is trial 0 with value: 0.4514864164705113.
[I 2023-10-10 22:06:48,876] A new study created in RDB with name: ANN


scaffold_splitter HLC ANN


[I 2023-10-10 22:06:49,342] Trial 0 finished with value: 2.7861343135035814 and parameters: {'learning_rate_init': 0.025107979675308954, 'hidden_layer_sizes': [5, 5, 5]}. Best is trial 0 with value: 2.7861343135035814.
